In [1]:
!pip install openpyxl



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import openpyxl


In [3]:
import sys
!{sys.executable} -m pip install openpyxl


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [4]:
import openpyxl
print("openpyxl is working!")


openpyxl is working!


In [5]:
import pandas as pd
import numpy as np

# Load original dataset 
df = pd.read_excel("C:/Users/shaik/Downloads/shipment_dataset_10000.xlsx")
df.head()


,order_id,supplier_id,supplier_rating,supplier_lead_time,order_date,promised_delivery_date,actual_delivery_date,shipment_mode,shipping_distance_km,order_quantity,unit_price,total_order_value,weather_condition,region,holiday_period,previous_on_time_rate,carrier_name,delayed_reason_code,on_time_delivery
0,1,5322,3.4,10,2024-05-15,2024-05-25,2024-05-29,Sea,51,48,2153.91,103387.68,Clear,East,No,89.9,EcomExpress,Operational,0
1,2,3932,4.3,10,2024-11-12,2024-11-22,2024-11-27,Sea,373,91,405.36,36887.76,Rainy,East,Yes,93.6,DHL,Customs,0
2,3,8966,3.2,5,2024-08-28,2024-09-02,2024-09-02,Sea,1304,25,3241.41,81035.25,Cloudy,South,No,78.2,BlueDart,NaN,1
3,4,9832,3.9,7,2024-08-12,2024-08-19,2024-08-19,Air,839,71,365.79,25971.09,Rainy,Central,No,85.2,FedEx,Operational,1
4,5,2126,3.2,8,2024-07-07,2024-07-15,2024-07-18,Sea,258,9,3052.84,27475.56,Cloudy,East,No,77.1,BlueDart,Customs,0


In [14]:
df_anom = df.copy()

# 1. Introduce Null values
for col in df_anom.columns[:3]:
    df_anom.loc[df_anom.sample(frac=0.01).index, col] = np.nan

# 2. Duplicate rows
duplicate_rows = df_anom.sample(50)
df_anom = pd.concat([df_anom, duplicate_rows], ignore_index=True)

# 3. Datatype mismatch (convert some numeric values to strings)
numeric_cols = df_anom.select_dtypes(include=[np.number]).columns[:2]

for col in numeric_cols:
    idx = df_anom.sample(30).index

    # Convert whole column to object before inserting strings
    df_anom[col] = df_anom[col].astype(object)

    df_anom.loc[idx, col] = df_anom.loc[idx, col].astype(str)


# 4. Outliers: Inflate values
if len(numeric_cols) > 0:
    outlier_idx = df_anom.sample(20).index
    df_anom.loc[outlier_idx, numeric_cols[0]] = pd.to_numeric(
        df_anom.loc[outlier_idx, numeric_cols[0]], errors='coerce'
    ) * 50

# 5. Logical errors
if "region" in df_anom.columns:
    df_anom.loc[df_anom.sample(30).index, "region"] = "Nest"

# Shipment date before order date
date_cols = [c for c in df_anom.columns if "date" in c.lower()]
if len(date_cols) >= 2:
    order_col, ship_col = date_cols[:2]
    idx = df_anom.sample(20).index
    df_anom.loc[idx, ship_col] = pd.to_datetime(df_anom.loc[idx, order_col]) - pd.Timedelta(days=3)

df_anom.head()


,order_id,supplier_id,supplier_rating,supplier_lead_time,order_date,promised_delivery_date,actual_delivery_date,shipment_mode,shipping_distance_km,order_quantity,unit_price,total_order_value,weather_condition,region,holiday_period,previous_on_time_rate,carrier_name,delayed_reason_code,on_time_delivery
0,1.0,5322.0,3.4,10,2024-05-15,2024-05-25,2024-05-29,Sea,51,48,2153.91,103387.68,Clear,Nest,No,89.9,EcomExpress,Operational,0
1,2.0,3932.0,4.3,10,2024-11-12,2024-11-22,2024-11-27,Sea,373,91,405.36,36887.76,Rainy,East,Yes,93.6,DHL,Customs,0
2,3.0,8966.0,3.2,5,2024-08-28,2024-09-02,2024-09-02,Sea,1304,25,3241.41,81035.25,Cloudy,South,No,78.2,BlueDart,NaN,1
3,4.0,9832.0,3.9,7,2024-08-12,2024-08-19,2024-08-19,Air,839,71,365.79,25971.09,Rainy,Central,No,85.2,FedEx,Operational,1
4,5.0,2126.0,3.2,8,2024-07-07,2024-07-15,2024-07-18,Sea,258,9,3052.84,27475.56,Cloudy,East,No,77.1,BlueDart,Customs,0


In [15]:
# Save as CSV with UTF-8 encoding
df_anom.to_csv("shipment_dataset_anomalous.csv", index=False, encoding='utf-8-sig')
print("Anomalous file saved as shipment_dataset_anomalous.csv")

Anomalous file saved as shipment_dataset_anomalous.csv


In [16]:
df_anom.to_excel("shipment_dataset_anomalous.xlsx", index=False)
print("Anomalous file saved as shipment_dataset_anomalous.xlsx")


Anomalous file saved as shipment_dataset_anomalous.xlsx


In [19]:
import pandas as pd
import numpy as np

# Load the anomalous dataset (CSV version)
df_clean = pd.read_csv("shipment_dataset_anomalous.csv", encoding="utf-8-sig")

print("Before cleaning:")
print(df_clean.info())
print(df_clean.head())

# 1. FIX NULL / MISSING VALUES

# Option 1: Forward fill, then backward fill
df_clean = df_clean.ffill().bfill()

# 2. REMOVE DUPLICATES

df_clean.drop_duplicates(inplace=True)


# 3. FIX DATATYPE MISMATCHES
#    (Some numeric columns contain strings because of anomaly)


for col in df_clean.columns:
    try:
        df_clean[col] = pd.to_numeric(df_clean[col])
    except:
        pass

# Re-run conversion for columns that still have object type but should be numeric
force_numeric_cols = []

for col in df_clean.columns:
    # Detect numeric-looking strings (e.g. "1234", "55.6")
    if df_clean[col].dtype == "object":
        if df_clean[col].astype(str).str.replace('.', '', 1).str.isdigit().any():
            force_numeric_cols.append(col)

for col in force_numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")


# 4. HANDLE OUTLIERS USING IQR CAPPING


numeric_cols = df_clean.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    # Cap values outside the range
    df_clean[col] = df_clean[col].clip(lower, upper)

# 5. FIX LOGICAL ERRORS - INVALID REGION NAMES

if "region" in df_clean.columns:
    # Assume the most frequent region is correct
    correct_region = df_clean["region"].mode()[0]
    
    # Replace invalid regions such as "Nest"
    df_clean.loc[~df_clean["region"].isin([correct_region]), "region"] = correct_region


# 6. FIX LOGICAL ERRORS - SHIPMENT DATE < ORDER DATE

# Find all date columns
date_cols = [col for col in df_clean.columns if "date" in col.lower()]

# Ensure date columns are proper datetime
for col in date_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

# Fix date order problem
if len(date_cols) >= 2:
    order_col, ship_col = date_cols[:2]   # first 2 date columns
    df_clean.loc[df_clean[ship_col] < df_clean[order_col], ship_col] = df_clean[order_col]


# SAVE CLEANED DATA

df_clean.to_csv("shipment_dataset_cleaned.csv", index=False, encoding="utf-8-sig")

print("\nCleaning complete! Saved as shipment_dataset_cleaned.csv")
print(df_clean.info())
df_clean.head()


Before cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10050 entries, 0 to 10049
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                9949 non-null   float64
 1   supplier_id             9950 non-null   float64
 2   supplier_rating         9950 non-null   float64
 3   supplier_lead_time      10050 non-null  int64  
 4   order_date              10050 non-null  object 
 5   promised_delivery_date  10050 non-null  object 
 6   actual_delivery_date    10050 non-null  object 
 7   shipment_mode           10050 non-null  object 
 8   shipping_distance_km    10050 non-null  int64  
 9   order_quantity          10050 non-null  int64  
 10  unit_price              10050 non-null  float64
 11  total_order_value       10050 non-null  float64
 12  weather_condition       10050 non-null  object 
 13  region                  10050 non-null  object 
 14  holiday_period       

,order_id,supplier_id,supplier_rating,supplier_lead_time,order_date,promised_delivery_date,actual_delivery_date,shipment_mode,shipping_distance_km,order_quantity,unit_price,total_order_value,weather_condition,region,holiday_period,previous_on_time_rate,carrier_name,delayed_reason_code,on_time_delivery
0,1.0,5322.0,3.4,10,2024-05-15,2024-05-25,2024-05-29,Sea,51,48,2153.91,103387.68,Clear,East,No,89.9,EcomExpress,Operational,0
1,2.0,3932.0,4.3,10,2024-11-12,2024-11-22,2024-11-27,Sea,373,91,405.36,36887.76,Rainy,East,Yes,93.6,DHL,Customs,0
2,3.0,8966.0,3.2,5,2024-08-28,2024-09-02,2024-09-02,Sea,1304,25,3241.41,81035.25,Cloudy,East,No,78.2,BlueDart,Customs,1
3,4.0,9832.0,3.9,7,2024-08-12,2024-08-19,2024-08-19,Air,839,71,365.79,25971.09,Rainy,East,No,85.2,FedEx,Operational,1
4,5.0,2126.0,3.2,8,2024-07-07,2024-07-15,2024-07-18,Sea,258,9,3052.84,27475.56,Cloudy,East,No,77.1,BlueDart,Customs,0


In [20]:
df_clean.to_csv(r"C:/Users/shaik/Desktop/shipment_dataset_cleaned.csv", 
                index=False, 
                encoding="utf-8-sig")


In [21]:
df_clean.to_csv(r"C:/Users/shaik/Desktop/shipment_dataset_anomalous.csv", 
                index=False, 
                encoding="utf-8-sig")
